In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from CV.preprocessing.ImageToArrayPreprocessor import ImageToArrayPreprocessor
from CV.preprocessing.aspectawarepreprocessor import AspectAwarePreprocessor
from CV.datasets.SimpleDatasetLoader import SimpleDatasetLoader
from CV.nn.conv.fcheadnet import FCHeadNet
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from imutils import paths
import numpy as np
import argparse
import os
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler

full = pd.read_csv('C:/Users/HOU/Documents/practical_bundle/origin/sugisakaya48de1-4.csv')
label= pd.read_csv('C:/Users/HOU/Documents/practical_bundle/origin/keyf0713.csv')

full = preprocessing.scale(full)
full[:3]
full = preprocessing.normalize(full, norm='l2')

'''aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")'''

# # grab the list of images that we’ll be describing, then extract
# # the class label names from the image paths
# print("[INFO] loading images...")
# imagePaths = list(paths.list_images("data3"))
# classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
# classNames = [str(x) for x in np.unique(classNames)]

# # initialize the image preprocessors
# aap = AspectAwarePreprocessor(128,128)
# iap = ImageToArrayPreprocessor()

# # load the dataset from disk then scale the raw pixel intensities to
# # the range [0, 1]

# sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
# (data, labels) = sdl.load(imagePaths, verbose = 500)
# data = data.astype("float")/255.0

# (trainX, testX, trainY, testY) = train_test_split(data, labels,
#     test_size=0.1, random_state=42)

# #convert the labels from integers to vectors
# #trainY = LabelBinarizer().fit_transform(trainY)
# #testY = LabelBinarizer().fit_transform(testY)

list=[]
n=0
for i in range(int(full.shape[0]/24)):
    for j in range(label.shape[0]):
        if (i)*0.5 <label.values[j] and ((i+1)*0.5)>label.values[j]:
            n=n+1
            #print(i,i*0.5,(i+1)*0.5,label.values[j])
            list.append(i)
            break;
    
            
data=[]
labels=[]
n=0
for i in range(int(full.shape[0]/24)):
    if i in list:
        data.append(full[i*24:(i+1)*24])
        labels.append(1)
    else:
        data.append(full[i*24:(i+1)*24])
        labels.append(0) 

data = np.array(data)

data = data.reshape(data.shape[0], 24, 24, 1)

(train_X, test_X, trainY, testY) = train_test_split(data,
                               labels, test_size=.2,random_state=27)

from keras.utils import to_categorical
lb = LabelBinarizer()
#train_y = lb.fit_transform(train_y)
#test_y = lb.fit_transform(test_y)
lb.fit(trainY)
trainY=lb.transform(trainY)
testY = lb.transform(testY)
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

# load the VGG16 network, ensuring the head FC layer sets are left
# off

baseModel = VGG16(weights="imagenet", include_top=False,
                 input_tensor=Input(shape=(24,24,3)))
#baseModel = VGG16(weights="imagenet", include_top=False)

# initialize the new head of the network, a set of FC layers
# followed by a softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames),256)

# place the head FC model on top of the base model -- this will
# become the actual model we will train
model = Model(inputs=baseModel.input, output = headModel)

# loop over all layers in the base model and freeze them so they
# will *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False
    
# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy",optimizer=opt,
             metrics=["accuracy"])

# train the head of the network for a few epochs (all other
# layers are frozen) -- this will allow the new FC layers to
# start to become initialized with actual "learned" values
# versus pure random
print("[INFO] training head...")
'''model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
    validation_data=(testX, testY), epochs=25,
    steps_per_epoch=len(trainX) // 32, verbose=1)'''
model.fit(trainX, trainY, batch_size=32,
    validation_data=(testX, testY), epochs=10,
     verbose=1)
# evaluate the network after initialization
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                           predictions.argmax(axis=1),target_names=classNames))


# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True
# 用小lr的sgd重新finetune
print("[INFO] re-compile model...")
opt = SGD(lr = 0.001)
model.compile(loss="categorical_crossentropy", optimizer = opt,
             metrics=["accuracy"])

print("[INFO] fine-tuning model...")
model.fit(trainX, trainY, batch_size=32,
    validation_data=(testX, testY), epochs=25,
     verbose=1)

# evaluate the network after fine-tune
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                           predictions.argmax(axis=1),target_names=classNames))

# save the model to disk
print("[INFO] serializing model...")
model.save("flowers17.model")

C:\Users\HOU\Anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\preprocessing\data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


0 0.0 0.5 [0.26]
3 1.5 2.0 [1.64]
4 2.0 2.5 [2.27]
7 3.5 4.0 [3.53]
8 4.0 4.5 [4.04]
11 5.5 6.0 [5.96]
12 6.0 6.5 [6.44]
13 6.5 7.0 [6.755]
16 8.0 8.5 [8.36]
24 12.0 12.5 [12.185]
31 15.5 16.0 [15.62]
36 18.0 18.5 [18.29]
38 19.0 19.5 [19.145]
39 19.5 20.0 [19.895]
40 20.0 20.5 [20.06]
41 20.5 21.0 [20.855]
42 21.0 21.5 [21.44]
43 21.5 22.0 [21.86]
45 22.5 23.0 [22.82]
48 24.0 24.5 [24.425]
51 25.5 26.0 [25.7]
52 26.0 26.5 [26.33]
54 27.0 27.5 [27.02]
55 27.5 28.0 [27.86]
56 28.0 28.5 [28.49]
57 28.5 29.0 [28.91]
61 30.5 31.0 [30.515]
64 32.0 32.5 [32.27]
72 36.0 36.5 [36.095]
78 39.0 39.5 [39.05]
79 39.5 40.0 [39.53]
80 40.0 40.5 [40.445]
81 40.5 41.0 [40.865]
82 41.0 41.5 [41.435]
83 41.5 42.0 [41.855]
84 42.0 42.5 [42.035]
85 42.5 43.0 [42.725]
86 43.0 43.5 [43.355]
87 43.5 44.0 [43.805]
88 44.0 44.5 [44.06]
90 45.0 45.5 [45.08]
91 45.5 46.0 [45.53]
92 46.0 46.5 [46.22]
95 47.5 48.0 [47.885]
96 48.0 48.5 [48.365]
98 49.0 49.5 [49.055]
99 49.5 50.0 [49.85]
105 52.5 53.0 [52.7]
107 53

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'block5_pool/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,512].